<a href="https://colab.research.google.com/github/Nunez350/DataScienceAndAnalytics/blob/main/Module_11_Optimization_PartThree/retail_pricing_model_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Analytics Edge of Retail Optimization

Both binary variables and non-linear functions


#gurobipy
The Gurobi Optimizer is a mathematical optimization software library for solving mixed-integer linear and quadratic optimization problems.

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 25.0 MB/s eta 0:00:00


# load require packages

In [2]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import plotly.express as px

### Lists

In [3]:
# Be careful with Python indices!
# The range function range(a,b) creates a range of integers starting at <a> but ending at <b-1>
weeks = list(range(157, 165))
print("weeks (t) ", weeks)
# Also note here that the first element of a list has index 0. The second element has index 1, and so on...
w1 = weeks[0] #denotes the number of the first week in our planning horizon
print(w1)
w2 = weeks[1] #denotes the number of the second week in our planning horizon
print(w2)

weeks (t)  [157, 158, 159, 160, 161, 162, 163, 164]
157
158


### Parameters

In [4]:
#From orginal price / sales(demand) data
# Developed linear demand model: used 27-156 weeks (3 years) of data; 27-104( years 1/2)Train, 105-156( last year)Test

# Variable: 𝑆𝑒𝑎𝑠𝑜𝑛X. Year divided int 13 seasons / 4 weeks per season (categorical variable)
# Variable: Price,  at time 𝑝_𝑡, 𝑝_(𝑡−1), 𝑝_(𝑡−2)
# Linear regression developed demand model (Constrain 1)
# Demand is characterized by our linearly additive model
# model: 𝑑_𝑡 = 2181 − 2801*𝑝_𝑡 + 929*𝑝_(𝑡−1) + 728*𝑝_(𝑡−2) −555.430*𝑆𝑒𝑎𝑠𝑜𝑛2 +...+ 949.056*𝑆𝑒𝑎𝑠𝑜𝑛13

# Store model parameters
intercept = 1.978242858
# causal factors
p_coeff = -2.809634145
p1_coeff = 0.963410728
p2_coeff = 0.759639170
# time series factor
season_coeff = {
    2: -0.562046910,
    3: 0.087545274,
    4: -0.402637480,
    5: -0.027326010,
    6: 0.004349885,
    7: -0.036102297,
    8: -0.069280527,
    9: 0.160276197,
    10: 1.104208897,
    11: 1.122711287,
    12: 1.176802194,
    13: 0.947945548
}


In [ ]:
# # Here we create a dictionary that associates a season with each week in the planning horizon
# season = {}
# for w in weeks:
#     season[w] = np.ceil((w % 52) / 4)

# print(season)

In [11]:





season = {}
for w in weeks:
    season[w] = ((w - 1) % 52) // 4 + 2  # Using integer division to assign seasons

print(season)
{157: 2, 158: 2, 159: 2, 160: 2, 161: 3, 162: 3, 163: 3, 164: 3}

{157: 2, 158: 2, 159: 2, 160: 2, 161: 3, 162: 3, 163: 3, 164: 3}


{157: 2, 158: 2, 159: 2, 160: 2, 161: 3, 162: 3, 163: 3, 164: 3}

## Model 1 - no price ladder or business rules
#### DECISION VARIABLES:  prices in weeks 157 – 169  and demand in weeks 157 – 169
######  Prices; 𝑝_𝑡  for  t = 157, … , 169   and   𝑑_𝑡  for  t = 157, … , 169
######  Demand; follows linearly additive model:
#### MAXIMIZE 𝑅𝑒𝑣𝑒𝑛𝑢𝑒= 𝑝157𝑑157 + 𝑝158𝑑158 + 𝑝159𝑑159 + ... + 𝑝169𝑑169
#### Objective function,  sales revenue in next quarter (weeks 157 – 169)

1.   List item
2.   List item


#### SUBJECT TO CONSTRAINTS:
#### Demand is characterized by our linearly additive model:
#### 𝑑_𝑡 = 2181 − 2801*𝑝_𝑡 + 929*𝑝_(𝑡−1) + 728*𝑝_(𝑡−2) −555.430*𝑆𝑒𝑎𝑠𝑜𝑛2 +...+ 949.056*𝑆𝑒𝑎𝑠𝑜𝑛13


In [12]:
# Create Gurobi model object - repository for all objects to be used in the model
mod1 = gp.Model ("price_model_1")

In [13]:
# Define decision variables
p = mod1.addVars(weeks, ub = 1)

In [14]:
p

{157: <gurobi.Var *Awaiting Model Update*>,
 158: <gurobi.Var *Awaiting Model Update*>,
 159: <gurobi.Var *Awaiting Model Update*>,
 160: <gurobi.Var *Awaiting Model Update*>,
 161: <gurobi.Var *Awaiting Model Update*>,
 162: <gurobi.Var *Awaiting Model Update*>,
 163: <gurobi.Var *Awaiting Model Update*>,
 164: <gurobi.Var *Awaiting Model Update*>}

In [15]:
# Set objective function
# Example first 2 lines fully written out,
# Short cut for writing out each line
#  after the first 2 weeks, the remaining weeks[2:], are developed using the sum() and a for loop  (weeks 159 on...)
#    price at week(wx) * demand (from linear demand fucntion at weekx)
# obj_fn = mod1.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
#                            p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
#                            sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
#                           GRB.MAXIMIZE)



obj_fn = mod1.setObjective(
    p[w1] * (intercept + p_coeff * p[w1] + p1_coeff * 1 + season_coeff[int(season[w1])]) +
    p[w2] * (intercept + p_coeff * p[w2] + p1_coeff * p[w1] + season_coeff[int(season[w2])]) +
    sum(p[w] * (intercept + p_coeff * p[w] + p1_coeff * p[w-1] + p2_coeff * p[w-2] + season_coeff[int(season[w])]) for w in weeks[2:]),
    GRB.MAXIMIZE
)


In [16]:
mod1.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 0 rows, 8 columns and 0 nonzeros
Model fingerprint: 0x649a0fc3
Model has 21 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 2e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolved: 0 rows, 8 columns, 0 nonzeros
Presolved model has 21 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 7
 AA' NZ     : 2.100e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.44793900e+05  1.691

In [17]:
p[157] # to see 1st prediction

<gurobi.Var C0 (value 0.6088470091840946)>

In [18]:
p[158] # to see 2nd prediction

<gurobi.Var C1 (value 0.562288899271462)>

In [19]:
# to see all predictions and save to a dataframe
df1 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df1.loc[w,"price"] = p[w].x
df1

,price
157,0.608847
158,0.562289
159,0.658145
160,0.688251
161,0.809346
162,0.803457
163,0.71753
164,0.59926


In [20]:
fig = px.line(df1, x=df1.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 2 - no business rules + price ladder

In [27]:
p_ladder = [1.00, 0.95, 0.85, 0.75, 0.60, 0.50] #create a list for price ladder

In [28]:
# Create model object
mod2 = gp.Model ("price_model_2")

In [29]:
# Define decision variables which includes ladder constraint
p = mod2.addVars(weeks)
x = mod2.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [30]:
# Set objective function
obj_fn = mod2.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [31]:
# Select price ladder value
constr_select_ladder = mod2.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod2.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)

In [32]:
mod2.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 56 columns and 104 nonzeros
Model fingerprint: 0x8fcd191b
Model has 21 quadratic objective terms
Variable types: 8 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 3e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5.4056188
Presolve removed 0 rows and 8 columns
Presolve time: 0.00s
Presolved: 16 rows, 48 columns, 88 nonzeros
Presolved model has 21 quadratic objective terms
Variable types: 8 continuous, 40 integer (40 binary)

Root relaxation: objective 6.126947e+00, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective 

In [33]:
df2 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df2.loc[w,"price"] = p[w].x
df2

,price
157,0.75
158,0.75
159,0.75
160,0.75
161,0.85
162,0.85
163,0.75
164,0.6


In [34]:
fig = px.line(df2, x=df2.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 3 - at most 4 promotions

In [35]:
# Create model object
mod3 = gp.Model ("price_model_3")

In [36]:
# Define decision variables
p = mod3.addVars(weeks)
x = mod3.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [37]:
# Set objective function
obj_fn = mod3.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [38]:
# Select price ladder value
constr_select_ladder = mod3.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod3.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)
# At most 4 promotions
constr_4_promo = mod3.addConstr(sum(x[w,k] for k in p_ladder[1:] for w in weeks) <= 4)

In [39]:
mod3.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 17 rows, 56 columns and 144 nonzeros
Model fingerprint: 0xc1a5ada9
Model has 21 quadratic objective terms
Variable types: 8 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 3e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 5.1195343
Presolve time: 0.00s
Presolved: 17 rows, 56 columns, 136 nonzeros
Presolved model has 21 quadratic objective terms
Variable types: 8 continuous, 48 integer (48 binary)

Root relaxation: objective 6.126359e+00, 43 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |

In [40]:
df3 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df3.loc[w,"price"] = p[w].x
df3

,price
157,0.85
158,0.85
159,1.0
160,1.0
161,1.0
162,1.0
163,0.75
164,0.6


In [41]:
fig = px.line(df3, x=df3.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 4 - full model

In [42]:
# Create model object
mod4 = gp.Model ("price_model_4")

In [43]:
# Define decision variables
p = mod4.addVars(weeks)
x = mod4.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [44]:
# Set objective function
obj_fn = mod4.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [45]:
# Select price ladder value
constr_select_ladder = mod4.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod4.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)
# At most 4 promotions
constr_4_promo = mod4.addConstr(sum(x[w,k] for k in p_ladder[1:] for w in weeks) <= 4)
# No consecutive promotions
constr_no_consec_promo = mod4.addConstrs((sum(x[w,k] for k in p_ladder[1:]) + sum(x[w+1,k] for k in p_ladder[1:]) <= 1) for w in weeks[:-1])

In [46]:
mod4.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 24 rows, 56 columns and 214 nonzeros
Model fingerprint: 0xa0baa59a
Model has 21 quadratic objective terms
Variable types: 8 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 3e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 4.7727766
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 23 rows, 56 columns, 166 nonzeros
Presolved model has 21 quadratic objective terms
Variable types: 8 continuous, 48 integer (48 binary)

Root relaxation: objective 6.092322e+00, 44 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective

In [47]:
df4 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df4.loc[w,"price"] = p[w].x
df4

,price
157,1.0
158,0.85
159,1.0
160,0.85
161,1.0
162,0.95
163,1.0
164,0.6


In [48]:
fig = px.line(df4, x=df4.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

### End of script
---
